In [164]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize, fixed_point
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import estimation_functions
# from integrated_market_model.market import IntegratedMarketModel

In [165]:
np.random.seed(100)

In [166]:
df = pd.read_csv(f'../data/market_integrates_1.csv')
print(df)

     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1         5.348061        20.964655  3.263354   
1             1         2         5.134796        20.155075  4.093757   
2             1         3         5.170203        20.191143  4.084675   
3             1         4         5.551272        20.831672  3.431042   
4             1         5         5.830341        20.934608  2.991747   
..          ...       ...              ...              ...       ...   
995         100         6         5.977387        20.225459  3.347218   
996         100         7         5.083662        20.315749  3.435301   
997         100         8         5.254944        20.117505  3.581924   
998         100         9         5.849048        20.945524  2.807978   
999         100        10         5.302210        20.055657  3.206264   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0         0.870393  0.023810  0.056976  3.749289

In [167]:
def solve_delta(theta, df, n_consumers, n_firms, time):

    v_p_all = np.random.normal(0, 1, n_consumers*time)
    # v_p_all = np.zeros(n_consumers*time)
    all_est_shares = np.zeros(time*n_firms)
    all_indirect_utilities = np.zeros(time*n_firms)

    for t in range(0, time):
        v_p = v_p_all[t*n_consumers:(t+1)*n_consumers]
        try1 = estimation_functions.simulate_market_shares_per_period(theta, df, v_p, n_consumers, n_firms, t)
        try2 = estimation_functions.get_indirect_utility(theta, df, n_firms, t)
        all_est_shares[t*n_firms:(t+1)*n_firms] = try1
        all_indirect_utilities[t*n_firms:(t+1)*n_firms] = try2

    
    return all_indirect_utilities + df.shares - all_est_shares

In [168]:
def get_indirect_utilities(theta, df, time, n_firms):
    all_indirect_utilities = np.zeros(time*n_firms)
    # all_estimated_shares = np.zeros(time*n_firms)
    for t in range(0, time):
        # v_p = np.random.normal(0, 1, n_consumers)
        indirect_utility_at_true_param = estimation_functions.get_indirect_utility(theta, df, n_firms, t)
        # estimated_share = estimation_functions.simulate_market_shares_per_period(theta_0, df, v_p, n_consumers, n_firms, t)

        # all_estimated_shares[t*n_firms:(t+1)*n_firms] = estimated_share
        all_indirect_utilities[t*n_firms:(t+1)*n_firms] = indirect_utility_at_true_param

    return all_indirect_utilities

# estimation_algorithm = solve_delta(theta_0, df, n_consumers, n_firms, time)

# print(all_indirect_utilities - estimation_algorithm)
# print(df.shares - all_estimated_shares)


In [169]:
def function_minimization(theta, df, time, n_firms, n_consumers):
    all_indirect_utilities = get_indirect_utilities(theta, df, time, n_firms)
    estimation_algorithm = solve_delta(theta, df, n_consumers, n_firms, time)

    return mean_squared_error(all_indirect_utilities, estimation_algorithm)


In [170]:
# theta_0 = np.array([2., -0.5, -0.3, 0.5, 1.])
theta_0 = np.array([1., 1., 1., 1., 1.])
time = 100
n_firms = 10
n_consumers = 500


minimization_values = []

for l in range(0, 5):
    res = minimize(function_minimization, theta_0, args=(df, time, n_firms, n_consumers))
    minimization_values.append(res.fun)

print(minimization_values)


[0.011475217234954432, 0.011486206222511472, 0.01147393487626474, 0.011469932339141266, 0.011487994165045307]
